In [10]:
import csv
import couchdb
import json
import folium

In [4]:
with open('filtered.json', 'r') as file:
    data = json.load(file)

In [20]:
#if you want to categorise in positive/negative
def sentiment_category(data):
    for i in range(len(data)):
        if  data[i]['sentiment'] <= -0.4:
            data[i]['category'] = 'negative'
        elif  data[i]['sentiment'] >= 0.4:
            data[i]['category'] = 'positive'
        else:
            data[i]['category'] = 'neutral'
sentiment_category(data)

In [8]:
desired_element = 'location'
unique_values = set()

for item in data:
    value = item.get(desired_element)
    if value:
        unique_values.add(value)

for value in unique_values:
    print(value)

Melville Forest, Victoria
Shoreham, Victoria
Ballarat, Victoria
Bendigo, Victoria
Melbourne, Victoria
Melton, Victoria
Castlemaine, Victoria
Warrnambool, Victoria
Torquay - Jan Juc, Victoria
Rushworth, Victoria
Pakenham, Victoria
Geelong, Victoria
Koroit, Victoria
Lorne, Victoria


In [22]:
with open('data_complete.json', 'r') as file:
    data = json.load(file)

desired_key = 'location'  
desired_element = 'Melbourne, Victoria'  

# Count the occurrences of the desired element
count = 0

for item in data:
    if desired_key in item and item[desired_key] == desired_element:
        count += 1

# Display the count
print(f"The element '{desired_element}' appears {count} times in the '{desired_key}' key.")

The element 'Melbourne, Victoria' appears 262 times in the 'location' key.


In [18]:
#convert to geojson
import csv
import json

with open('suburbs_import.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    coordinates_data = list(csvreader)

with open('filtered.json', 'r') as file:
    data = json.load(file)

for item in data:
    location_name = item.get('location')
    if location_name:
        # Find the matching coordinates in the CSV
        for row in coordinates_data:
            if location_name == row[0]:
                latitude = row[1]
                longitude = row[2]
                item['latitude'] = latitude
                item['longitude'] = longitude
                break  # Stop searching once a match is found

with open('data_with_coordinates.json', 'w') as file:
    json.dump(data, file, indent=4)

In [45]:
# to align with Sa4 codes
with open('data_complete.json', 'r') as file:
    data = json.load(file)
    

for item in data:
    location = item.get('location')
    if location:
        modified_location = location.replace(", Victoria", "")

        if modified_location == "Melbourne":
            modified_location = modified_location + " - Inner"

        item['location'] = modified_location

# Save the modified data to a new JSON file
with open('modified_data.json', 'w') as file:
    json.dump(data, file, indent=4)

In [25]:

import geojson

with open('data_complete.json', 'r') as file:
    data = json.load(file)

# Create a GeoJSON feature collection
features = []

for item in data:
    location = item.get('location')
    text = item.get('text')
    sentiment = item.get('sentiment')
    category = item.get('category')
    longitude = float(item.get('longitude'))
    latitude = float(item.get('latitude'))

    if location and text and sentiment and category and longitude and latitude:
        point = geojson.Point((longitude, latitude))
        properties = {
            'location': location,
            'text': text,
            'sentiment': sentiment,
            'category': category
        }
        feature = geojson.Feature(geometry=point, properties=properties)
        features.append(feature)

feature_collection = geojson.FeatureCollection(features)

# Save the GeoJSON to a file
with open('output.geojson', 'w') as file:
    geojson.dump(feature_collection, file)

In [26]:
with open('output.geojson', 'r') as file:
    geoJSON = json.load(file)

In [42]:
map = folium.Map(location = [-37.814, 144.96332], tiles = "Stamen Terrain", zoom_start = 10)
c = folium.Choropleth(
    geo_data = geoJSON,
    name = 'Sentiment of Public Transport in Victoria',
    data = geoJSON,
    columns = ['sentiment', 'location'],
)
c.add_to(map)
map